Source: [D2L section 3.2](https://d2l.ai/chapter_linear-networks/linear-regression-scratch.html)


In [92]:
#!pip install d2l==0.17.0
%matplotlib inline
import random
import numpy as np
import tensorflow as tf
# from d2l import tensorflow as d2l
# for easier reading np
np.set_printoptions(precision=4,suppress=True)

**Generating the Dataset**

In [93]:
with open('iris.csv', 'r', encoding='utf-8-sig') as f:
    temp = np.genfromtxt(f,delimiter=',')
temp = np.array(temp, dtype=np.float32)
features = temp[:,:3]
labels = temp[:,3]

In [94]:
print('features:', features[0], '\nlabel:', labels[0])
# d2l.set_figsize()
# The semicolon is for displaying the plot only
# d2l.plt.scatter(features[:, (1)].numpy(), labels.numpy(), 1);

features: [5.1 3.5 1.4] 
label: 0.2


**Reading the Dataset**

In [95]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = tf.constant(indices[i:min(i + batch_size, num_examples)])
        yield tf.gather(features, j), tf.gather(labels, j)

In [96]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tf.Tensor(
[[5.8 2.8 5.1]
 [5.8 2.6 4. ]
 [6.3 2.8 5.1]
 [6.6 3.  4.4]
 [5.4 3.9 1.7]
 [6.4 3.2 5.3]
 [4.6 3.1 1.5]
 [4.9 3.1 1.5]
 [7.6 3.  6.6]
 [6.7 3.1 4.4]], shape=(10, 3), dtype=float32) 
 tf.Tensor([2.4 1.2 1.5 1.4 0.4 2.3 0.2 0.1 2.1 1.4], shape=(10,), dtype=float32)


**Initializing Model Parameters**

In [97]:
w = tf.Variable(tf.random.normal(shape=(3, 1), mean=0, stddev=0.01),
                trainable=True)
b = tf.Variable(tf.zeros(1), trainable=True)

**Defining the Model**

In [98]:
def linreg(X, w, b): 
    """The linear regression model."""
    return tf.matmul(X, w) + b

**Defining the Loss Function**

In [99]:
def squared_loss(y_hat, y):
    """Squared loss."""
    return (y_hat - tf.reshape(y, y_hat.shape))**2 / 2

**Defining the Optimization Algorithm**

In [100]:
def sgd(params, grads, lr, batch_size):
    """Minibatch stochastic gradient descent."""
    for param, grad in zip(params, grads):
        param.assign_sub(lr * grad / batch_size)

**Training**

In [101]:
lr = 0.03
num_epochs = 5
net = linreg
loss = squared_loss

In [102]:
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        with tf.GradientTape() as g:
            l = loss(net(X, w, b), y)  # Minibatch loss in `X` and `y`
        # Compute gradient on l with respect to [`w`, `b`]
        dw, db = g.gradient(l, [w, b])
        # Update parameters using their gradient
        sgd([w, b], [dw, db], lr, batch_size)
    train_l = loss(net(features, w, b), labels)
    print(f'epoch {epoch + 1}, loss {float(tf.reduce_mean(train_l)):f}')

epoch 1, loss 0.143614
epoch 2, loss 0.031784
epoch 3, loss 0.047673
epoch 4, loss 0.096483
epoch 5, loss 0.024490


In [103]:
print(w)
print(b)
from sklearn.metrics import r2_score
y_hat = [b + w[0]*y_row[0] + w[1]*y_row[1] + w[2]*y_row[2] for y_row in features]
print(r2_score(labels, y_hat))

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[ 0.0149],
       [-0.1038],
       [ 0.3808]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([-0.0247], dtype=float32)>
0.9153366789053365


In [104]:
from sklearn.linear_model import LinearRegression
print(features.shape)
print(labels.shape)

reg = LinearRegression().fit(features,labels)

print('intercept= ', reg.intercept_)
print('Slope (coefficients)=', reg.coef_)
print('r-squared', reg.score(features, labels))

(150, 3)
(150,)
intercept=  -0.24872482
Slope (coefficients)= [-0.2103  0.2288  0.5261]
r-squared 0.9380481272962801
